DGX A100 best practices notes
=============================

## 基本的な発想

DGX A100はDockerで使う、というのが基本的な発想のようです。

NVIDIA GPU Cloud (NGC)というレポジトリに、NVIDIAマシン用にチューンされたディープラーニングフレームワーク＋学習モデルのDockerイメージがあり、そこから使いたいものを選んで使うことで、マシンの性能を一番引き出せるとのこと。

イメージも頻繁にバージョンアップするので、開発中はイメージ自体はいじらず、自分のコードやデータセットは別ディレクトリに用意してコンテナにバインドして使うことが推奨されています。もちろん既存イメージの上にライブラリを足さなければいけない場合などは自分でイメージを作ります。コードがある程度固まってきたらイメージに組み込んでパッケージ化することも考えられます。

DGXシステムを購入したことで、NGC上にプライベートスペースが作られており、そこで自分たちが作成したイメージを共有できるようになっています。NGCを使うにはNVIDIAアカウントを作る必要がありますが、個人用アカウントを作って後からuticeppというorganizationに足すこともできますし、最初からuticeppのメンバーとしてアカウントを作ることもできます。使用したい（すでに使用している）メールアドレスを飯山に伝えてください。

NGC: https://ngc.nvidia.com/signin

## 具体例

NGCにログインし、右上のユーザーアイコンからSetupをクリック。Generate API Keyの画面からAPIキーを発行させたあと、
```
$ docker login nvcr.io
Username: $oauthtoken
Password: <API key>

$ docker pull nvcr.io/nvidia/tensorflow:20.11-tf2-py3
$ docker run --gpus all -it --rm -v $HOME:$HOME -v /raid/datasets:/data nvcr.io/nvidia/tensorflow:20.11-tf2-py3 bash
```
でTFコンテナが使えます。`docker login`は一度だけやれば以降は必要ありません。`$oauthtoken`は何かの環境変数ではなく、そのまま書き込んでください。

## DockerとGPU

最新のdockerコマンドはホストマシンのGPUドライバがコンテナの中から見られるようになっています。コンテナの作成時に
```
docker run --gpus all #use all GPUs
docker run --gpus 4   #use GPUs number 0 to 3
docker run --gpus '"device=<comma-separated GPU UUID or GPU number>"' #use specific GPUs
```
とGPUを指定します。GPUのレンジ（#0からではなく）を指定したり、複数の特定のGPUを選ぶ方法は未確認です。GPUのUUIDは`/proc/driver/nvidia/gpus/<ID>/information`から見られます。`device=`で指定するときは上の通りにクオートをつける必要があるようです。

GPUドライバ（NVIDIA Driver 455.45.01など）はカーネルスペースの住人なので、コンテナからドライバのバージョンを選ぶということはできません。しかし、CUDAドライバ（libcuda.so）とCUDAツールキット（libcudart.soなど）はユーザースペースにあるので、コンテナごとにバージョンを選べます。CUDAのそれぞれのバージョンは特定のバージョン以降のGPUドライバで使用可能なので、いつでも古いCUDAバージョンのコンテナを使うことができます。詳しくは[NVIDIA docs](https://docs.nvidia.com/deploy/cuda-compatibility/index.html)を参照。

## NVIDIAイメージのバージョン

NVIDIAが組んだディープラーニングフレームワーク（TF, PyTorch, etc.）のイメージには共通のバージョンタグがついており、CUDAなどのバージョンが縦管理されています。

https://docs.nvidia.com/deeplearning/frameworks/support-matrix/index.html